In [1]:
import pandas as pd, numpy as np
from db_queries import get_ids, get_outputs, get_location_metadata, get_population, get_covariate_estimates
from get_draws.api import get_draws
import scipy.stats 
import scipy.integrate as integrate
import matplotlib.pyplot as plt

The purpose of this notebook is to create generalized/customizable functions that can be used for Large Scale Food Fortification multiplication models with dichotomous outcomes (zinc, vitamin A, folic acid). The outcomes (DALYs averted) generated by this notebook assume the following:

- Complete scale-up achieved between starting baseline and alternative scenario coverage (med/high/low levels), defined according to the proportion of the population that eats industrially produced vehicles. This notebook does NOT currently consider the additional coverage over time in the alternative scenario defined according to the proportion of the population that eats the vehicle at all (due to campaign to convince additional individuals to eat fortified versions of vehicle).
- All individuals covered by fortification are assumed to be *effectively* covered. This assumption is not valid based on age- and timing-effects built into the full-scale models. These nutrient-specific effects should be added into the respective mutliplication model for the full results

In [2]:
location_ids = [163, 214, 205, 190, 189]

"""Note: full set of location IDs is shown below, but subset used here
was selected because they are the locations with non-missing coverage data
for the nutrient and vehicle of interest (vitamin A/oil)

[168, 161, 201, 202, 6, 205, 171, 141, 179, 207, 163, 11, 180, 181,
184, 15, 164, 213, 214, 165, 196, 522, 190, 189, 20]"""

ages = [2,3,4,5]
sexes = [1,2]

index_cols=['location_id','sex_id','age_group_id']

# define alternative scenario coverage levels (low, medium, high)
    # this parameter represents the proportion of additional coverage achieved in the
    # alternative scenario, defined as the difference between the proportion of the population
    # that eats the fortified vehicle and the proportion of the population that eats 
    # the industrially produced vehicle
alternative_scenario_coverage_levels = [0.25, 0.5, 0.75]

In [3]:
# vitamin A specific -- these should be replaced for other models
rei_id = 96
cause_ids = [389, 302, 341]
nonfatal_causes = [389]
nutrient = 'vitamin a'
vehicle = 'oil'

In [4]:
# define no fortification relative risk distribution
# vitamin a specific -- this should be replaced for other models

from numpy import log
from scipy.stats import norm, lognorm

# median and 0.975-quantile of lognormal distribution for RR
median = 2.22
q_975 = 5.26

# 0.975-quantile of standard normal distribution (=1.96, approximately)
q_975_stdnorm = norm().ppf(0.975)

mu = log(median) # mean of normal distribution for log(RR)
sigma = (log(q_975) - mu) / q_975_stdnorm # std dev of normal distribution for log(RR)

# Frozen lognormal distribution for RR, representing uncertainty in our effect size
# (s is the shape parameter)
rr_distribution = lognorm(s=sigma, scale=median)

In [5]:
def generate_fortification_rr_draws_lognormal_dist(mean, std):
    """This function takes a distribution for the relative risk
    for lack of fortification of a particular nutrient and generates
    1,000 draws based on that distribution. The data is the duplicated
    so that it is the same for each location ID so that it can be easily
    used later in the calculations."""
    data = pd.DataFrame()    
    np.random.seed(343)
    data['rr'] = np.random.lognormal(mean, std, size=1000)
    draws = []
    for i in list(range(0,1000)):
        draws.append(f'draw_{i}')
    data['draws'] = draws
    data = pd.DataFrame.pivot_table(data, values='rr', columns='draws').reset_index().drop(columns=['index'])
    df = pd.DataFrame(np.repeat(data.values,len(location_ids),axis=0))
    df.columns = data.columns
    df['location_id'] = location_ids
    df = df.set_index('location_id')
    return df

In [6]:
def pull_gbd_pafs(rei_id, cause_ids):
    """This function pulls PAF data from GBD for specified 
    risk outcome pairs. Note that the risk in this context 
    will/should be nutrient *deficiencies*, not the lack of 
    nutrient fortification"""
    
    data = pd.DataFrame()
    for cause_id in cause_ids:
        temp = get_draws(
            gbd_id_type=['rei_id', 'cause_id'], 
            gbd_id=[rei_id, cause_id],
            source='burdenator',
            measure_id=2, #dalys
            metric_id=2, #percent
            location_id=location_ids,
            year_id=2019,
            age_group_id=ages,
            sex_id=sexes, 
            gbd_round_id=6,
            status='best',
            decomp_step='step5',
        )
        data = pd.concat([data,temp], ignore_index=True)
    data = data.set_index(index_cols + ['cause_id'])
    data = data.drop(columns=[c for c in data.columns if 'draw' not in c]).sort_index()
    return data

In [7]:
def pull_gbd_dalys(cause_ids):
    """This function pulls dalys for specified cause IDs from GBD"""
    
    ylds = get_draws(
        gbd_id_type='cause_id', 
        gbd_id=cause_ids,
        source='como',
        measure_id=3,
        metric_id=3, # only available as rate
        location_id=location_ids,
        year_id=2019,
        age_group_id=ages,
        sex_id=sexes, 
        gbd_round_id=6,
        status='best',
        decomp_step='step5',
    ).set_index(index_cols + ['cause_id'])
    ylds = ylds.drop(columns=[c for c in ylds.columns if 'draw' not in c])
    pop = get_population(
        location_id=location_ids,
        year_id=2019,
        age_group_id=ages,
        sex_id=sexes,
        gbd_round_id=6,
        decomp_step='step4').set_index(index_cols)
    for i in list(range(0,1000)):
        ylds[f'draw_{i}'] = ylds[f'draw_{i}'] * pop['population']
    ylls = get_draws(
        gbd_id_type='cause_id', 
        gbd_id=cause_ids,
        source='codcorrect',
        measure_id=4,
        metric_id=1, 
        location_id=location_ids,
        year_id=2019,
        age_group_id=ages,
        sex_id=sexes, 
        gbd_round_id=6,
        status='latest',
        decomp_step='step5',
    ).set_index(index_cols + ['cause_id']).replace(np.nan, 0)
    ylls= ylls.drop(columns=[c for c in ylls.columns if 'draw' not in c])
    for nf in nonfatal_causes:
        nonfatal = ylls.groupby(index_cols).sum()
        nonfatal['cause_id'] = nf
        for i in list(range(0,1000)):
            nonfatal[f'draw_{i}'] = 0
    ylls = pd.concat([ylls.reset_index(), nonfatal.reset_index()]).set_index(index_cols + ['cause_id'])
    
    dalys = ylls + ylds
    return dalys

In [66]:
def load_coverage_data(nutrient, vehicle):
    data = pd.read_csv('/ihme/homes/alibow/notebooks/vivarium_data_analysis/pre_processing/lsff_project/data_prep/outputs/LSFF_extraction_clean_data_rich_locations_01_11_2021.csv')
    alpha = (data.loc[data.vehicle == vehicle]
             .loc[data.nutrient == nutrient]
             .loc[data.value_description == 'percent of population eating fortified vehicle'])
    alpha_star = (data.loc[data.vehicle == vehicle]
                  .loc[data.value_description == 'percent of population eating industrially produced vehicle'])

    
    # generate draws
    """This currently relies on two major assumptions:
    1. Truncated normal distribution
    2. The same percentile from the eats_fortified and eats_fortifiable distributions sampled for each draw
    
    Assumption number two is likely overly restrictive, but was chosen such that eats_fortified will 
    always be less than eats_fortifiable at the draw level (this is consistent with methodology described
    in 2017 concept model, but is achieved by setting the same random seed to sample each of these
    parameters)"""
      
    for data in [alpha, alpha_star]:
              
        data['value_std'] = (data.value_975_percentile - data.value_025_percentile) / 2 / 1.96
        data['a'] = (data.value_025_percentile - data.value_mean) / data.value_std
        data['b'] = (data.value_975_percentile - data.value_mean) / data.value_std       
        np.random.seed(1246)
        for i in list(range(0,1000)):
            data[f'draw_{i}'] = scipy.stats.truncnorm.rvs(data.a, data.b, data.value_mean, data.value_std) / 100
            
    alpha = (alpha.set_index('location_id')
         .drop(columns=[c for c in alpha.columns if 'draw' not in c and c != 'location_id']))
    alpha_star = (alpha_star.set_index('location_id')
         .drop(columns=[c for c in alpha_star.columns if 'draw' not in c and c != 'location_id']))
    alpha_star_low = (alpha_star - alpha) * alternative_scenario_coverage_levels[0] + alpha
    alpha_star_low['coverage_level'] = 'low'
    alpha_star_med = (alpha_star - alpha) * alternative_scenario_coverage_levels[1] + alpha
    alpha_star_med['coverage_level'] = 'medium'
    alpha_star_high = (alpha_star - alpha) * alternative_scenario_coverage_levels[2] + alpha
    alpha_star_high['coverage_level'] = 'high'
    
    alpha_star = pd.concat([alpha_star_low.reset_index(), 
                            alpha_star_med.reset_index(), 
                            alpha_star_high.reset_index()], 
                           ignore_index=True)
    alpha_star = alpha_star.set_index([c for c in alpha_star.columns if 'draw' not in c])
    
    p = 1 - alpha
    p_star = 1 - alpha_star
    
    return p, p_star

In [9]:
def calculate_fortification_paf(fortification_rrs, p):
    """This function calculates the population attributable fraction of UNfortified food
    on the fortification outcome of interest (outcome defined in the fortification 
    effect size, which is generally nutrient deficiency)
    
    NOTE: this function does not consider age/time lags of fortification effects
    (assumes that every individual covered by fortification is effectively covered)"""
       
    fort_paf = ((fortification_rrs - 1) * p) / ((fortification_rrs - 1) * (p + 1))    
    return fort_paf

In [10]:
def calculate_pif(fort_paf, p, p_star):
    """This function calculates the population impact fraction for UNfortified 
    foods and nutrient deficiency based on the location-specific coverage
    levels of fortified foods; specifically, p (1 - proportion of population
    that eats fortified vehicle) and p_start (1 - proportion of population that 
    eats industrially produced vehicle).
    
    NOTE: this function does not consider age/time lags of fortification effects
    (assumes that every individual covered by fortification is effectively covered)"""
    pif = fort_paf * (p - p_star) / p
    return pif

In [22]:
def calculate_daly_reduction_by_cause(pif, pafs, dalys):
    """This functionc calculates the population impact fraction for UNfortified 
    food and DALYs due to specific causes as well as the total number of DALYs
    averted by cause, sex, and age
    
    NOTE: this function does not consider age/time lags of fortification effects
    (assumes that every individual covered by fortification is effectively covered)"""
    
    df = pd.DataFrame()
    
    for level in ['low','medium','high']:
        pif_level = (pif.reset_index()
                     .loc[pif.reset_index().coverage_level == level]
                     .drop(columns='coverage_level')
                     .set_index('location_id'))
        pif_dalys = pif_level * pafs
        pif_dalys['measure'] = 'pif'
        dalys_reduction = pif_dalys * dalys
        dalys_reduction['measure'] = 'dalys averted'
        dalys_reduction_overall = dalys_reduction.reset_index().groupby(index_cols + ['measure']).sum().reset_index()
        dalys_reduction_overall['cause_id'] = 294
        data = (pd.concat([pif_dalys.reset_index(), dalys_reduction.reset_index(), dalys_reduction_overall], ignore_index=True))
        data['coverage_level'] = level
        data = data.set_index(index_cols + ['measure','cause_id','coverage_level']).dropna().sort_index()
        df = pd.concat([df,data])
        
    return df

In [67]:
fort_deficiency_rrs = generate_fortification_rr_draws_lognormal_dist(mu, sigma)
gbd_pafs = pull_gbd_pafs(rei_id, cause_ids)
dalys = pull_gbd_dalys(cause_ids)
p, p_star = load_coverage_data(nutrient, vehicle)
fort_deficiency_paf = calculate_fortification_paf(fort_deficiency_rrs, p)
fort_deficiency_pif = calculate_pif(fort_deficiency_paf, p, p_star)
fort_daly_reduction = calculate_daly_reduction_by_cause(fort_deficiency_pif, gbd_pafs, dalys)

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v104/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [68]:
# check and make sure that there are only negative dalys averted for execpted draws
    # (draws with RR for fortification < 1 and draws with negative GBD PAFs)

in_neg_draws = np.concatenate([pd.DataFrame(fort_deficiency_rrs.stack()).loc[pd.DataFrame(fort_deficiency_rrs.stack())[0] < 1].reset_index()['draws'].unique(),
            pd.DataFrame(gbd_pafs.stack()).loc[pd.DataFrame(gbd_pafs.stack())[0] < 0].reset_index()['level_4'].unique()])

out_neg_draws = pd.DataFrame(fort_daly_reduction.stack()).reset_index().rename(columns={'level_6':'draw',0:'val'})
out_neg_draws = out_neg_draws.loc[out_neg_draws.val < 0]

assert len([c for c in out_neg_draws.draw.unique() if c not in in_neg_draws]) == 0, "Error: unexpected negative values"

In [69]:
fort_daly_reduction_by_location = fort_daly_reduction.groupby(['location_id','measure','cause_id','coverage_level']).sum().reset_index()
fort_daly_reduction_by_location = (fort_daly_reduction_by_location
                                   .loc[fort_daly_reduction_by_location.measure=='dalys averted']
                                   .loc[fort_daly_reduction_by_location.cause_id==294])
fort_daly_reduction_by_location = (fort_daly_reduction_by_location
                                   .set_index(['location_id','measure','cause_id','coverage_level'])
                                   .apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1))
    
fort_daly_reduction_by_location

count          mean  \
location_id measure       cause_id coverage_level                         
163         dalys averted 294      high            1000.0  69633.656512   
                                   low             1000.0  23211.218837   
                                   medium          1000.0  46422.437675   
189         dalys averted 294      high            1000.0   7545.163174   
                                   low             1000.0   2515.054391   
                                   medium          1000.0   5030.108782   
190         dalys averted 294      high            1000.0   7257.985482   
                                   low             1000.0   2419.328494   
                                   medium          1000.0   4838.656988   
205         dalys averted 294      high            1000.0      0.000000   
                                   low             1000.0      0.000000   
                                   medium          1000.0      0.000000   
214         dalys averted 294      high            1000.0  21582.833833   
                                   low             1000.0   7194.277944   
                                   medium          1000.0  14388.555888   

                                                            std          min  \
location_id measure       cause_id coverage_level                              
163         dalys averted 294      high            21182.516951  3354.346363   
                                   low              7060.838984  1118.115454   
                                   medium          14121.677967  2236.230908   
189         dalys averted 294      high             3684.105084 -2475.625731   
                                   low              1228.035028  -825.208577   
                                   medium           2456.070056 -1650.417154   
190         dalys averted 294      high             4130.561112 -2191.300189   
                                   low              1376.853704  -730.433396   
                                   medium           2753.707408 -1460.866793   
205         dalys averted 294      high                0.000000     0.000000   
                                   low                 0.000000     0.000000   
                                   medium              0.000000     0.000000   
214         dalys averted 294      high            10287.101631 -4609.538526   
                                   low              3429.033877 -1536.512842   
                                   medium           6858.067754 -3073.025684   

                                                           2.5%           50%  \
location_id measure       cause_id coverage_level                               
163         dalys averted 294      high            32986.620181  68485.906767   
                                   low             10995.540060  22828.635589   
                                   medium          21991.080120  45657.271178   
189         dalys averted 294      high             2321.637806   6963.271813   
                                   low               773.879269   2321.090604   
                                   medium           1547.758538   4642.181209   
190         dalys averted 294      high             1451.733771   6406.459741   
                                   low               483.911257   2135.486580   
                                   medium            967.822514   4270.973160   
205         dalys averted 294      high                0.000000      0.000000   
                                   low                 0.000000      0.000000   
                                   medium              0.000000      0.000000   
214         dalys averted 294      high             4791.562258  20166.618229   
                                   low              1597.187419   6722.206076   
                                   medium           3194.374838  13444.412153   

                                     